# decoding

use the model to predict some new cards!

In [6]:
import numpy as np
from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, Activation, Lambda
from keras.callbacks import ModelCheckpoint
import h5py

In [7]:
# restrict GPU usage here
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [8]:
# read in data
c2i = np.load('data/c2i.npy').item()
i2c = np.load('data/i2c.npy').item()
ycards = np.load('data/ycards.npy')

## hyperparameters

although dropout, batch size and epochs aren't used at decode, we need the variables that define the model size.

In [10]:
# copied from training
DROP_RATE = 0.00

HIDDEN_SIZE = 500             # lstm feature vector size
MAX_Y_LEN = ycards.shape[1]   # maximum card length
VOCAB_SIZE = len(c2i.keys())  # number of characters

## decoding

In [11]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_input  = Input(shape=(MAX_Y_LEN, ), name='lm_input')
decoder_embed  = Embedding(VOCAB_SIZE, HIDDEN_SIZE, 
                           mask_zero=True, trainable=True, name='lm_emb')
decoder_lstm1  = LSTM(HIDDEN_SIZE, 
                      return_sequences=True, 
                      return_state=True, 
                      name='lm_lstm1')
decoder_lstm2  = LSTM(HIDDEN_SIZE, 
                      return_sequences=True, 
                      return_state=True, 
                      name='lm_lstm2')

decoder_dense_1  = Dense(HIDDEN_SIZE, activation='relu', name='lm_dns_1')

def weight_tying(layer_input):
    result = K.dot(layer_input, K.transpose(decoder_embed.weights[0]))
    return result

decoder_dense_2 = Lambda(weight_tying, name='weight_tying')
decoder_dense_3 = Activation('softmax')

x = decoder_embed(decoder_input)
x = Dropout(DROP_RATE)(x)
x, h1, c1 = decoder_lstm1(x)
x = Dropout(DROP_RATE)(x)
x, h2, c2 = decoder_lstm2(x)
x = Dropout(DROP_RATE)(x)
x = decoder_dense_1(x)
x = Dropout(DROP_RATE)(x)
x = decoder_dense_2(x)
x = decoder_dense_3(x)

model = Model(decoder_input, x)

In [12]:
model.load_weights('model/weights_tiedfinal.h5')

In [13]:
# this input is for the previously-predicted character
decoder_input  = Input(shape=(1, ))
# these inputs are the recurrent states
decoder_state_input_h1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c1 = Input(shape=(HIDDEN_SIZE,))
decoder_states_inputs1 = [decoder_state_input_h1, decoder_state_input_c1]
decoder_state_input_h2 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c2 = Input(shape=(HIDDEN_SIZE,))
decoder_states_inputs2 = [decoder_state_input_h2, decoder_state_input_c2]

# we reuse the embedding layer
x = decoder_embed(decoder_input)
x, dh1, dc1 = decoder_lstm1(x, initial_state=decoder_states_inputs1)
decoded_states1 = [dh1, dc1]
x, dh2, dc2 = decoder_lstm2(x, initial_state=decoder_states_inputs2)
decoded_states2 = [dh2, dc2]

x = decoder_dense_1(x)
x = decoder_dense_2(x)
x = decoder_dense_3(x)

In [14]:
gen_model = Model(inputs=[decoder_input] + decoder_states_inputs1 + decoder_states_inputs2, 
                  outputs=[x] + decoded_states1 + decoded_states2)

In [15]:
gen_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
lm_emb (Embedding)              multiple             51000       input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 500)          0                                            
__________________________________________________________________________________________________
lm_lstm1 (

## decode function

we initialize the states randomly, and start our sequence qith the SOS character. until we reach a set length or we reach an end-of-sequence character, we will generate a probability distribution over the next predicted characters, sample a character randomly according to the distribution (we won't use a greedy or beam-search method because we *want* a degree of 'wackiness' in this case), and input that character (along with the LSTM previous states) *back* into the model to generate another character, etc.

the *temperature* scales the softmax distribution, allowing for more or less randomness in the network predictions. a temperature of 1 is unscaled, a temperature above one means that the relative probabilities are closer (and thus the network is more 'random'), while temperatures below 1 make the network more confident (and thus more 'conservative').

In [16]:
def decode_sequence(temperature=1.0, maxlen=256, seed=None, debug=False):
    # randomize input state vectors.
    a = np.random.random(HIDDEN_SIZE).reshape(1, -1)
    b = np.random.random(HIDDEN_SIZE).reshape(1, -1)
    c = np.random.random(HIDDEN_SIZE).reshape(1, -1)
    d = np.random.random(HIDDEN_SIZE).reshape(1, -1)
    states1 = [a, b]
    states2 = [c, d]
    decoded_sentence = []
    # Generate empty target sequence of length 1.
    # Populate the first character of target sequence with the start character.
    # add seed if present
    target_seq = [c2i['Ⓢ']]
    if seed is not None:
        seed = seed.lower()
        if seed[-1] != '⒞':
            seed += '⒞'
        for char in seed:
            target_seq.append(c2i[char])
            
    # pre-load seed - run loop without saving output
    # and use target output instead; akin to teacher forcing
    if len(target_seq) > 1:
        for i in range(len(target_seq)-1):
            output_tokens, h1, c1, h2, c2 = gen_model.predict([np.array([target_seq[i]])] + states1 + states2)
            states1 = [h1, c1]
            states2 = [h2, c2]
        for c in seed:
            decoded_sentence.append(c)
                
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    while not stop_condition:
        if debug:
            print('inp:', [np.array([target_seq[-1]])])
            print('st1:', np.shape(states1))
            print('st2:', np.shape(states2))
            
        output_tokens, h1, c1, h2, c2 = gen_model.predict([np.array([target_seq[-1]])] + states1 + states2)
        if debug:
            # print('typ:', type(output_tokens), type(h1), type(c2), type(h2), type(c2))
            print('out:', output_tokens.shape)
            print('max:', i2c[target_seq[-1]], '=>', i2c[np.argmax(output_tokens)])
            print()
        
        # Update states
        states1 = [h1, c1]
        states2 = [h2, c2]
        
        def sample(a, temperature=temperature):
            a = np.array(a)**(1/temperature)
            p_sum = a.sum()
            sample_temp = a/p_sum 
            # stupid fix for > 1 error
            while sum(sample_temp) > 1:
                sample_temp[0] -= 0.0001
            return np.argmax(np.random.multinomial(1, sample_temp, 1))
        
        # Sample a token with temperature
        sampled_token_index = idx = sample(np.squeeze(output_tokens))
        sampled_char = i2c[sampled_token_index]
        decoded_sentence.append(sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if len(decoded_sentence) > maxlen*2 or sampled_char in ['Ⓔ']:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq.append(sampled_token_index)

    return decoded_sentence

## generate extension idea:

you could allow users to 'seed' any number of fields by eliciting the targets. e.g. the user wants to make themselves as a card, so they choose:  
`name` = "billy bob"  
`type` = "Legendary Creature - Human"

during the generation loop, when the leading tag for that field is generated, e.g. `⒯` for `type`, then just like the current code, the *desired* input is 'forced' into the LSTM until the end of that field (so for type we would end the sequence with `⒫` to signal the *end* of the type).

In [17]:
# Ⓢ for name (at start of card)
# ⒞ for mana cost
# ⒭ for rarity
# ⒯ for type & subtype
# ⒫ for power & toughness
# ⒜ for each ability

In [18]:
def generate(temperature=1, seed=None):
    card = ''.join(decode_sequence(temperature=temperature, seed=seed)).replace('Ⓔ', '')
    rarity = {'S': 'promo', 'M': 'mythic', 'C': 'common', 'U': 'uncommon', 'R': 'rare'}
    splits = ['⒞', '⒭', '⒯', '⒫', '⒜']
    for s in splits:
        card = card.replace(s, '|'+s)
    card = card.split('|')
    
    cardd = {}
    cardd['abil'] = []
    for i, l in enumerate(card):
        if i == 0:
            cardd['name'] = card[i].title()
        else:
            l = l.replace('Ⓝ', cardd['name'])
            if '⒞' in l:
                cardd['cost'] = l.replace('⒞', '')
            elif '⒭' in l:
                cardd['rare'] = rarity.get(l.replace('⒭', ''), 'unknown')
            elif '⒯' in l:
                cardd['type'] = l.replace('⒯', '').replace(':', ': ').replace('·', ' ')
            elif '⒫' in l:
                if 'creature' in cardd['type']:
                    cardd['pt'] = l.replace('⒫', '')
            else:
                cardd['abil'].append(l.replace('⒜', '').replace(' x ', ' X '))
    
    for cat in ['name', 'cost', 'rare', 'type', 'abil', 'pt']:
        if cat in cardd.keys():
            if cat == 'abil':
                for a in cardd[cat]:
                    print(a)
            else:
                print(cardd[cat])
    return cardd

## examples

here we generate some cards with different temperature settings

i did cheat here to generate a 'well-formed' card

In [36]:
c = generate(temperature=0.25)

Colanatic Spirit
③Ⓦ
common
creature: elf shaman
whenever Colanatic Spirit deals combat damage to a player, you may pay ①. if you do, target creature gets +2/+2 until end of turn.
1/1


In [46]:
c = generate(temperature=1.0)

Odremon Imp
②Ⓖ
uncommon
creature: vampire
②Ⓖ: Odremon Imp gains protection from the color of your choice until end of turn.
2/1


In [70]:
c = generate(temperature=1.5)

Oatodra-Hey Dranrer
④Ⓤ
uncommon
creature: chamber spirit
lifelink
4/6


In [74]:
c = generate(temperature=3)

Puy Awl U6Pl, Bjoyrkva
⑪ⓊⓊⓊ
creature: elfrakaun julcuklcihaPuy Awl U6Pl, Bjoyrkva
R
RⓇⓊ: buflova Puy Awl U6Pl, Bjoyrkva extun
3/3


## with some name seeds

In [75]:
c = generate(temperature=1, seed='Mark Rosewater')

Mark Rosewater
②Ⓦ
common
creature: dryad

2/2


In [80]:
c = generate(temperature=1, seed='Richard Garfield')

Richard Garfield
Ⓖ
uncommon
creature: djinn
flying
5/5


In [85]:
c = generate(temperature=1, seed='Gavin Verhey')

Gavin Verhey
②Ⓤ
common
creature: human assassin
when Gavin Verhey enters the battlefield, sacrifice Gavin Verhey.
2/3


In [78]:
c = generate(temperature=1, seed='Melissa DeTora')

Melissa Detora
④ⒷⒷ
promo
creature: screat soldier

2/2


In [89]:
c = generate(temperature=1, seed='Gaby Spartz')

Gaby Spartz
③Ⓖ
rare
creature: cat guard
sacrifice Gaby Spartz: draw a card.
3/3
